In [31]:
import os
import pandas as pd

In [32]:
# 데이터 병합
base_path = '../data/vote/'
city = os.listdir(base_path)
if '.DS_Store' in city:
    city.remove('.DS_Store')
city.sort()
print(city)

['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']


In [33]:
# 선거 결과 파일 병합

df = pd.DataFrame()
for c in city:
    files = os.listdir(os.path.join(base_path, c))
    
    if '.DS_Store' in files:
        files.remove('.DS_Store')
        
    for file in files:
        csv = os.path.join(base_path, c, file)
        try:
            temp = pd.read_csv(csv, header=1)

            temp['읍면동명'] = temp['읍면동명'].fillna(method='ffill')
            temp.loc[temp['투표구명'].isna(), '투표구명'] = temp.loc[temp['투표구명'].isna()]['읍면동명']
            idx = temp.iloc[0,4:-3].sort_values(ascending=False).index

            temp.rename(columns={idx[0]:'당선 득표'}, inplace='True')
            temp.rename(columns={idx[1]:'차선 득표'}, inplace='True')

            if idx[0][0] == '미':
                temp['당선 정당'] = '미래통합당'
                temp['당선 이름'] = idx[0].replace('미래통합당', '')
            elif idx[0][0] == '더':
                temp['당선 정당'] = '더불어민주당'
                temp['당선 이름'] = idx[0].replace('더불어민주당', '')
            elif idx[0][0] == '정':
                temp['당선 정당'] = '정의당'
                temp['당선 이름'] = idx[0].replace('정의당', '')
            elif idx[0][0] == '무':
                temp['당선 정당'] = '무소속'
                temp['당선 이름'] = idx[0].replace('무소속', '')

            if idx[1][0] == '미':
                temp['차선 정당'] = '미래통합당'
                temp['차선 이름'] = idx[1].replace('미래통합당', '')
            elif idx[1][0] == '더':
                temp['차선 정당'] = '더불어민주당'
                temp['차선 이름'] = idx[1].replace('더불어민주당', '')
            elif idx[1][0] == '정':
                temp['차선 정당'] = '정의당'
                temp['차선 이름'] = idx[1].replace('정의당', '')
            elif idx[1][0] == '무':
                temp['차선 정당'] = '무소속'
                temp['차선 이름'] = idx[1].replace('무소속', '')
            elif idx[1][0] == '민':
                temp['차선 정당'] = '민생당'
                temp['차선 이름'] = idx[1].replace('민생당', '')
            temp['기타'] = temp[idx[3:]].sum(axis=1)
            temp['선거구명']=file.split('.')[0].split('_')[0]
            if c[-2:] in ['남도','북도']:
                temp['시도명'] = c[0]+c[2]
            else:
                temp['시도명'] = c[:2]
            temp['선거구명 전체'] = temp['시도명'] + ' ' + temp['선거구명']
            
            temp = temp[['선거구명 전체', '시도명', '선거구명', '읍면동명', '투표구명', '선거인수', '투표수', '당선 정당', '당선 이름', '당선 득표',  '차선 정당', '차선 이름', '차선 득표', '기타', '계', '무효투표수', '기권수']]
    
            df = pd.concat([df, temp]).reset_index(drop=True)
        except Exception as e:
            print('error', e, csv)
df.head()

,선거구명 전체,시도명,선거구명,읍면동명,투표구명,선거인수,투표수,당선 정당,당선 이름,당선 득표,차선 정당,차선 이름,차선 득표,기타,계,무효투표수,기권수
0,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,합계,합계,58270,39729,미래통합당,이철규,20685,더불어민주당,김동완,17373,700.0,39279,450,18541
1,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,거소·선상투표,거소·선상투표,178,166,미래통합당,이철규,95,더불어민주당,김동완,57,10.0,164,2,12
2,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,관외사전투표,관외사전투표,3447,3447,미래통합당,이철규,1342,더불어민주당,김동완,1861,92.0,3368,79,0
3,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,국외부재자투표,국외부재자투표,71,21,미래통합당,이철규,2,더불어민주당,김동완,18,0.0,21,0,50
4,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,국외부재자투표(공관),국외부재자투표(공관),0,0,미래통합당,이철규,0,더불어민주당,김동완,0,0.0,0,0,0


In [34]:
# 선거구확인 (253건)
len(df['선거구명 전체'].unique()), df['선거구명 전체'].unique()

(253,
 array(['강원 동해시태백시삼척시정선군', '강원 속초시인제군고성군양양군', '강원 춘천시철원군화천군양구군을',
        '강원 홍천군횡성군영월군평창군', '강원 춘천시철원군화천군양구군갑', '강원 원주시을', '강원 강릉시',
        '강원 원주시갑', '경기 광주시갑', '경기 김포시을', '경기 수원시정', '경기 고양시병', '경기 의정부시갑',
        '경기 부천시병', '경기 포천시가평군', '경기 시흥시갑', '경기 성남시수정구', '경기 용인시정',
        '경기 군포시', '경기 용인시병', '경기 화성시갑', '경기 여주시양평군', '경기 이천시', '경기 파주시을',
        '경기 광명시을', '경기 안산시상록구을', '경기 시흥시을', '경기 의왕시과천시', '경기 하남시',
        '경기 평택시갑', '경기 용인시을', '경기 광주시을', '경기 안산시상록구갑', '경기 김포시갑',
        '경기 부천시정', '경기 안양시만안구', '경기 남양주시갑', '경기 화성시을', '경기 구리시', '경기 수원시갑',
        '경기 용인시갑', '경기 파주시갑', '경기 고양시정', '경기 고양시을', '경기 의정부시을', '경기 남양주시병',
        '경기 부천시갑', '경기 동두천시연천군', '경기 화성시병', '경기 안성시', '경기 평택시을', '경기 양주시',
        '경기 고양시갑', '경기 수원시을', '경기 안양시동안구갑', '경기 부천시을', '경기 안양시동안구을',
        '경기 광명시갑', '경기 안산시단원구을', '경기 수원시무', '경기 오산시', '경기 성남시분당구을',
        '경기 성남시중원구', '경기 수원시병', '경기 성남시분당구갑', '경기 안산시단원구갑', '경기 남양주시을',
        '경남 사천시남해군하동군', '경남 밀양시의령군함안군창녕군', '경남 김해시갑', '경남 통영시고성군',
        

In [35]:
df.to_csv('../data/korea_map_21th_election_result.csv', index=False)

# 데이터 분석

In [37]:
election = pd.read_csv('../data/korea_map_21th_election_result.csv')
election.head()

,선거구명 전체,시도명,선거구명,읍면동명,투표구명,선거인수,투표수,당선 정당,당선 이름,당선 득표,차선 정당,차선 이름,차선 득표,기타,계,무효투표수,기권수
0,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,합계,합계,58270,39729,미래통합당,이철규,20685,더불어민주당,김동완,17373,700.0,39279,450,18541
1,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,거소·선상투표,거소·선상투표,178,166,미래통합당,이철규,95,더불어민주당,김동완,57,10.0,164,2,12
2,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,관외사전투표,관외사전투표,3447,3447,미래통합당,이철규,1342,더불어민주당,김동완,1861,92.0,3368,79,0
3,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,국외부재자투표,국외부재자투표,71,21,미래통합당,이철규,2,더불어민주당,김동완,18,0.0,21,0,50
4,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,국외부재자투표(공관),국외부재자투표(공관),0,0,미래통합당,이철규,0,더불어민주당,김동완,0,0.0,0,0,0


In [44]:
election['득표차'] = election['당선 득표'] - election['차선 득표']
election['당선 득표 비율'] = round(election['당선 득표'] / election['계'] * 100, 2)
election['차선 득표 비율'] = round(election['차선 득표'] / election['계'] * 100, 2)
election.fillna(0, inplace=True)
election.head()

,선거구명 전체,시도명,선거구명,읍면동명,투표구명,선거인수,투표수,당선 정당,당선 이름,당선 득표,차선 정당,차선 이름,차선 득표,기타,계,무효투표수,기권수,득표차,당선 득표 비율,차선 득표 비율
0,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,합계,합계,58270,39729,미래통합당,이철규,20685,더불어민주당,김동완,17373,700.0,39279,450,18541,3312,52.66,44.23
1,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,거소·선상투표,거소·선상투표,178,166,미래통합당,이철규,95,더불어민주당,김동완,57,10.0,164,2,12,38,57.93,34.76
2,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,관외사전투표,관외사전투표,3447,3447,미래통합당,이철규,1342,더불어민주당,김동완,1861,92.0,3368,79,0,-519,39.85,55.26
3,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,국외부재자투표,국외부재자투표,71,21,미래통합당,이철규,2,더불어민주당,김동완,18,0.0,21,0,50,-16,9.52,85.71
4,강원 동해시태백시삼척시정선군,강원,동해시태백시삼척시정선군,국외부재자투표(공관),국외부재자투표(공관),0,0,미래통합당,이철규,0,더불어민주당,김동완,0,0.0,0,0,0,0,0.00,0.00


In [46]:
# 
sub = election[election['투표구명'] =='합계'][['선거구명 전체', '무효투표수', '득표차', '당선 득표 비율', '차선 득표 비율']]
sub

,선거구명 전체,무효투표수,득표차,당선 득표 비율,차선 득표 비율
0,강원 동해시태백시삼척시정선군,450,3312,52.66,44.23
75,강원 속초시인제군고성군양양군,397,184,48.14,47.11
116,강원 동해시태백시삼척시정선군,286,3053,54.77,41.99
162,강원 춘천시철원군화천군양구군을,511,391,49.92,48.65
202,강원 춘천시철원군화천군양구군을,249,163,49.90,48.63
...,...,...,...,...,...
22999,충북 청주시상당구,1041,3025,47.10,43.98
23081,충북 청주시서원구,1400,3334,49.86,46.79
23160,충북 보은군옥천군영동군괴산군,410,3975,57.57,40.53
23206,충북 증평군진천군음성군,254,883,51.75,47.02


In [55]:
s1 = sub.groupby('선거구명 전체')[['득표차', '무효투표수']].sum().sort_values('득표차')
s1['결과'] = s1['득표차'] <  s1['무효투표수']
s1[s1['결과'] == True]

,득표차,무효투표수,결과
선거구명 전체,,,
인천 동구미추홀구을,171,1451,True
충남 아산시갑,564,995,True
부산 사하구갑,697,1239,True
서울 용산구,890,1400,True
경기 성남시분당구갑,1128,1368,True
충남 천안시갑,1328,1760,True
경남 창원시진해구,1405,2195,True
